# `04-memory`

# Memtory (대화 내용 기억)
 
- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다 (stateless)
- 이전 대화 내용을 계속 프롬프트에 주입해야 함

1. Short-Term Memory
    - 단기기억: 한 대화 세션에 대한 기억
2. Long-Term Memory
    - 장기기억: 전체 세션에서 추출한 중요한 정보

## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어 넣는 것.
2. 이 때 대화가 길어지면 토큰 수 증가 및 성능 하락이 일어남
3. 개선 방식 컨셉
    1. 요약
    2. 적정 길이에서 앞부분 자르기
    3. 정리(특정 명사들로 정리, Node-Edge 그래프 방식 등)

## ConverstionBufferMemory

- 메시지 저장 -> 변수에서 추출 가능

In [17]:

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

{'chat_history': []}

In [18]:
from operator import itemgetter
# `chat_history`변수에, load_memory_var 결과를 저장하고, `chat_history 키를 추출`
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

In [19]:
chain = runnable | prompt | llm 

In [20]:
res = chain.invoke({'input': '만나서 반가워'})

In [21]:
memory.save_context(
    {'human': '만나서 반가워'},
    {'ai': res.content}
)

In [22]:
my_message = '내 이름 뭐라고?'
res = chain.invoke({'input': my_message})
memory.save_context(
    {'human': my_message},
    {'ai': res.content}
)

In [23]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반가워', additional_kwargs={}, response_metadata={}),
  AIMessage(content='저도 만나서 반가워요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내 이름 뭐라고?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송하지만, 아직 당신의 이름을 모르겠어요. 이름을 알려주시면 기억해둘게요!', additional_kwargs={}, response_metadata={})]}

In [25]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)


# Prompt
prompt = ChatPromptTemplate(
    [
        ('system', '넌 좀 틱틱대는 챗봇이야. '),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

chain = runnable | prompt | llm | StrOutputParser()

# 메세지 스트리밍 하는 함수
def stream_msg(chain, msg):
    full_msg = ''
    print('AI: ', end='')
    for token in chain.stream({'input': msg}):
        full_msg += token
        print(token, end='', flush=True)
    print('\n---')
    return full_msg

input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()
    print('인간: ', input_msg)
    output_msg = stream_msg(chain, input_msg)
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg}
    )
    time.sleep(0.5)

인간:  난 데이터 분석가야
AI: 아, 데이터 분석가시구나. 어떤 프로젝트나 데이터에 대해 이야기하고 싶어?
---
인간:  너가 아이디어 좀 줘봐
AI: 아이디어를 원하시는군요. 어떤 분야나 목적에 맞는 데이터 분석 아이디어를 찾고 계신가요? 예를 들어, 고객 분석, 판매 예측, 소셜 미디어 분석 등 구체적인 분야를 알려주시면 더 적합한 아이디어를 드릴 수 있어요.
---
인간:  카페
AI: 카페 관련 데이터 분석 아이디어를 원하시는군요. 좀 더 구체적으로 어떤 목표를 가지고 싶은지 알려주시면 좋겠어요. 예를 들어, 고객 행동 분석, 매출 증대 방안, 메뉴 최적화, 고객 만족도 향상 등 어떤 방향이든 맞춰서 아이디어를 드릴 수 있어요.
---
인간:  카페 상권
AI: 카페 상권 분석이라니, 꽤 흥미로운 주제군요. 어떤 구체적인 목표가 있나요? 예를 들어, 신규 매장 입지 선정, 경쟁 분석, 고객 유입 패턴 파악, 매출 예측 등. 그에 맞는 데이터 분석 아이디어를 제시해 드릴게요.
---
인간:  그만할래
AI: 알겠어요. 필요하면 언제든 다시 이야기하자고요.
---
인간:  그만
AI: 알겠습니다. 언제든 필요하시면 말씀하세요.
---


In [27]:
# 레포트 작성 챗봇
from operator import itemgetter
from pprint import pprint
from pydantic import BaseModel, Field

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# Output parser 정의
class Report(BaseModel):
    title: str = Field(..., description='보고서의 제목')
    summary: str = Field(..., description='보고서 요약본')
    content_kr: str = Field(..., description='한국어로 작성된 보고서의 내용(1000자 이내)')
    content_jp: str = Field(..., description='영어로 작성된 보고서의 내용(1000자 이내)')

parser = PydanticOutputParser(pydantic_object=Report)

# Prompt에 format instructions 추가
prompt = ChatPromptTemplate(
    [
        ('system', '넌 보고서 작성에 특화된 챗봇이야. '
                   '반드시 지정된 형식에 맞춰 JSON으로 답변해.\n\nFORMAT INSTRUCTION: {format_instructions}'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

# 체인 구성
chain = runnable | prompt | llm | parser

input_msg = ''

# 사용자가 ('quit', '정지', '그만', '') 중에 하나를 입력하면 대화 종료
while 1:
    input_msg = input()
    if input_msg  in ('quit', '정지', '그만', ''):
        break

    print('인간: ', input_msg)
    output_msg = chain.invoke({'input': input_msg})
    pprint(output_msg.model_dump())
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg.model_dump_json()}
    )